<a href="https://colab.research.google.com/github/Disha-Sikka/Dog-Breed-Classification/blob/main/dogs_breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle competitions download -c ssd-3-l-dog-breed-classification


ssd-3-l-dog-breed-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref= zipfile.ZipFile('/content/ssd-3-l-dog-breed-classification.zip')
zip_ref.extractall('/content')
zip_ref.close()